In [1]:
# import the necessary packages
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense

In [2]:
'''
class SimpleCNN:
    @staticmethod
    def build(width, height, depth, classes):
        # initialize the model along with the input shape
        model = Sequential()
        inputShape = (height, width, depth)
        chanDim = -1

        # first CONV => RELU => BN layer set
        model.add(Conv2D(32, (3, 3), strides=(2, 2), padding="same",input_shape=inputShape))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))

        # second CONV => RELU => BN layer set
        model.add(Conv2D(64, (3, 3), strides=(2, 2), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))

        # first (and only) set of FC => RELU layers
        model.add(Flatten())
        model.add(Dense(128))
        model.add(Activation("relu"))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))

        # softmax classifier
        model.add(Dense(classes))
        model.add(Activation("softmax"))

        # return the constructed network architecture
        return(model)
'''

In [2]:
# The model that you will implement should have the following architecture
#1 -> Convolutional layer with 6 kernels and tanh activation function
#2 -> Max pooling downsampling the image by 2
#3 -> Convolutional layer with 16 kernels and tanh activation function
#4 -> Max pooling downsampling the image by 2
#5 -> Fully connected layer with 120 units
#6 -> Fully connected layer with 84 units
#7 -> Fully connected layer with 10 units

class SimpleCNN:
    @staticmethod
    def build(width, height, depth, classes):
    # Demo Model Definition
    inp = tf.keras.Input(shape=(28,28,1))

    l1 = tf.keras.layers.Conv2D(filters=6, kernel_size=3, activation='tanh')(inp)
    l2 = tf.keras.layers.MaxPooling2D(pool_size=(2,2))(l1)
    l3 = tf.keras.layers.Conv2D(filters=16, kernel_size=3, activation='tanh')(l2)
    # Each kernel is a 3*3 matrix = (kernel_size*kernel_size matrix). However, there as 6 channels as inputs.
    # Therefore, each filter is in reality composed of 6 kernels and thus is a 6*3*3 matrix. 
    # Hence we get 6*3*3+1 (+1=bias) = 55 coeff per filter.
    # As we have 16 filters, we get 16*55=880 coeff.

    l4 = tf.keras.layers.MaxPooling2D(pool_size=(2,2))(l3)

    x = tf.keras.layers.Flatten()(l4)
    # We flatten a 5*5*16 D object, we get a 400D vector.

    l5 = tf.keras.layers.Dense(120, activation='tanh')(x)
    # Dense from 400D to 120D + 120 bias = 401*120 = 48120 coeff.
    l6 = tf.keras.layers.Dense(84, activation='tanh')(l5)
    out = tf.keras.layers.Dense(10, activation='softmax')(l6)

    model = tf.keras.Model(inputs=inp, outputs=out)
    model.summary()
    return(model)


IndentationError: expected an indented block (<ipython-input-2-ee3e7a1c18db>, line 14)

In [3]:
# import the necessary packages
from tensorflow.keras.losses import MSE
from tensorflow.keras.losses import CategoricalCrossentropy as CCE
import tensorflow as tf

In [4]:
def generate_image_adversary(model, image, label, eps=2 / 255.0):
    # cast the image
    image = tf.cast(image, tf.float32)
    
    # record our gradients
    with tf.GradientTape() as tape:
        # explicitly indicate that our image should be tacked for gradient updates
        tape.watch(image)

        # use our model to make predictions on the input image and then compute the loss
        pred = model(image)
        loss = CCE(label, pred)
        
        # calculate the gradients of loss with respect to the image, then compute the sign of the gradient
        gradient = tape.gradient(loss, image)
        signedGrad = tf.sign(gradient)

        # construct the image adversary
        adversary = (image + (signedGrad * eps)).numpy()

        # return the image adversary to the calling function
        return(adversary)

In [5]:
# import the necessary packages
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
import numpy as np
import cv2

In [6]:
# load MNIST dataset and scale the pixel values to the range [0, 1]
print("[INFO] loading MNIST dataset...")
(trainX, trainY), (testX, testY) = mnist.load_data()
trainX = trainX / 255.0
testX = testX / 255.0

# add a channel dimension to the images
trainX = np.expand_dims(trainX, axis=-1)
testX = np.expand_dims(testX, axis=-1)

# one-hot encode our labels
trainY = to_categorical(trainY, 10)
testY = to_categorical(testY, 10)

[INFO] loading MNIST dataset...


In [ ]:
# initialize our optimizer and model
print("[INFO] compiling model...")
opt = Adam(lr=1e-3)
model = SimpleCNN.build(width=28, height=28, depth=1, classes=10)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

# train the simple CNN on MNIST
print("[INFO] training network...")
model.fit(trainX, trainY, validation_data=(testX, testY), batch_size=64, epochs=5, verbose=1)

In [ ]:
# make predictions on the testing set for the model trained on
# non-adversarial images
(loss, acc) = model.evaluate(x=testX, y=testY, verbose=0)
print("[INFO] loss: {:.4f}, acc: {:.4f}".format(loss, acc))

In [ ]:
model.save("model_12_02_2022")

In [7]:
from tensorflow.keras.models import load_model

In [8]:
model = load_model("model_12_02_2022")

OSError: SavedModel file does not exist at: model_12_02_2022/{saved_model.pbtxt|saved_model.pb}

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
n_adv = 4

# f, axarr = plt.subplots(n_adv, figsize=(50, 50))
fig, axs = plt.subplots(n_adv, 2, figsize=(500,500), constrained_layout=True)

# loop over a sample of our testing images
num = 0
for i in np.random.choice(np.arange(0, len(testX)), size=(n_adv,)):
    # grab the current image and label
    image = testX[i] ; label = testY[i]
    pred = model.predict(image.reshape((1,28,28,1)))

    # generate an image adversary for the current image and make
    # a prediction on the adversary
    adversary = generate_image_adversary(model, image.reshape(1, 28, 28, 1), label, eps=0.1)
    adv_pred = model.predict(adversary)
    adv_image = adversary.reshape((28,28))
    
    axs[num,0].imshow(image.reshape((28,28)), cmap='gray')
    axs[num,0].get_xaxis().set_visible(False) ; axs[num,0].get_yaxis().set_visible(False)
    axs[num,0].set_title("IMG : {:.0f} | PRED : {:.0f} ({:.2f}%)".format(i,np.argmax(pred),np.max(pred)*100))
    
    axs[num,1].imshow(adv_image, cmap='gray')
    axs[num,1].get_xaxis().set_visible(False) ; axs[num,1].get_yaxis().set_visible(False)
    axs[num,1].set_title("ADV_IMG : {:.0f} | PRED : {:.0f} ({:.2f}%)".format(i,np.argmax(adv_pred),np.max(adv_pred)*100))
    num += 1